# Valeur foncière moyenne



In [4]:
#import all the libraries needed
import statistics
import requests 
import json
import math

In [1]:
rayon = 50
lat = 48.85 #latitude
lon = 2.35 #longitude

def valeur_fonciere_au_m2(rayon, lat, lon):

    params = {"format": "json"}
    response = requests.get(f"https://api.cquest.org/dvf?lat={lat}&lon={lon}&dist={rayon}", params=params) #requête de l'API

    def jprint(obj):
        # création d'un string à partir de l'objet JSON
        text = json.dumps(obj, indent=4)
        print(text)

    valeur_fonciere_moyenne = [] # tableau où on stocke les valeurs foncières se trouvant dans le périmètre choisi

    valeur_api = response.json() #la valeur foncière que l'on cherche avec l'API

    for i in range(len(valeur_api["features"])): 
        surface = (valeur_api["features"][i]["properties"]["surface_relle_bati"]) #Recherche de surface du logement
        if surface != 0 :
            prix = (valeur_api["features"][i]["properties"]["valeur_fonciere"]) #Recherche de la valeur foncière
            prix_au_m2 = prix/surface
            valeur_fonciere_moyenne.append(prix_au_m2)

    return round(statistics.mean(valeur_fonciere_moyenne))

In [3]:
#print(f"La valeur foncière moyenne sur un rayon de {rayon}m est de {valeur_fonciere_au_m2(rayon, lat, lon)} euros par m2.\n")

#  Coûts de construction

## Estimation peu fiable

In [4]:
def cout_construction_immeuble1(surface, lon, lat): 
    assert(surface >= 0)

    surface += surface/10 #surface totale = surface logement + partie commune ≈ 10%

    prix_loyer = valeur_fonciere_au_m2(20, lon, lat)*surface #Exemple rayon de 20m pour le calcul de valeur_fonciere_au_m2
    
    prix_etudes_de_sol = prix_loyer*0.02
    prix_controles_techniques = prix_loyer*0.03
    prix_honoraires_architectes = prix_loyer*0.08
    return round(prix_loyer + prix_etudes_de_sol + prix_controles_techniques + prix_honoraires_architectes)

## Estimation plus fiable 

# Coûts de destruction 


In [14]:
def calculer_cout_demolition(surface, nombre_etages, indice_complexite, indice_etat): #les indices sont entre 1-3
    # Coûts initiaux par unité de surface
    cout_par_metre_carre = 100  # Coût de base par mètre carré
    
    # Facteurs multiplicateurs en fonction de la complexité et de l'état du bâtiment
    facteur_complexite = [1, 1.2, 1.5]  # Indice de complexité (1-3)
    facteur_etat = [1, 1.1, 1.3]  # Indice d'état (1-3)
    
    # Calcul des coûts de base
    cout_base = surface * cout_par_metre_carre
    
    # Calcul des coûts en fonction de la complexité et de l'état du bâtiment
    cout_total = cout_base * facteur_complexite[indice_complexite-1] * facteur_etat[indice_etat-1]
    
    # Coûts supplémentaires en fonction du nombre d'étages
    cout_supplementaire_etages = 5000 * nombre_etages
    
    # Ajout des coûts supplémentaires
    cout_total += cout_supplementaire_etages
    
    return round(cout_total)


In [22]:
# Demande des informations à l'utilisateur

surface = 200     #surface du bâtiment cumulée  (en mètres carrés)
nombre_etages = 2      # nombre d'étages du bâtiment)
indice_complexite = 2  #indice de complexité du bâtiment (de indice_standing à 3)
indice_etat = 1        #indice de l'état du bâtiment (de indice_standing à 3)

# Appel de la fonction pour calculer les coûts de démolition
cout_demolition = calculer_cout_demolition(surface, nombre_etages, indice_complexite, indice_etat)

# Affichage du résultat
print("Le coût de démolition du bâtiment est de", cout_demolition, "euros.")

Le coût de démolition du bâtiment est de 32080 euros.


# Coûts de réhabilitation 

In [9]:
# Problèmes de construction
geotechnical_investigation_prices = {
    "étude topographique": 2.16,  # € par mètre carré
    "rapport ingénieur": 2100  # € par jour 
}

demolition_prices = {
    "lavabo": 51,  # € chacun
    "baignoire": 82,  # € chacune
    "douche": 137,  # € chacune
    "lampe incandescente": 17,  # € chacune
    "lave-vaisselle": 31,  # € chacun
    "porte": 55,  # € chacune
    "fenêtre": 56,  # € chacune
    "poutre": 16.40,  # € par mètre linéaire
    "cloison en plâtre": 4.95,  # € par mètre carré
    "charpente": 11.08,  # € par mètre linéaire
    "tuiles": 15.50,  # € par mètre carré
    "murs": 43.92,  # € par mètre carré
    "murs porteurs": 174.04,  # € par mètre carré
    "sol en béton": 48.70,  # € par mètre carré
    "location camion hydraulique 12T": 1400  # € par jour
}

# Dégats des eaux
mold_prices = {
    "inspection de la zone": 481,  # €
    "réduction de la moisissure": 1923,  # €
    "enlèvement de zones contaminées par la moisissure": 18132  # €
}

mold_demolition_prices = {
    "sol": 3.29,  # € par pied carré (10.79 € par mètre carré)
    "plafond": 2.17  # € par pied carré (7.12 € par mètre carré)
}

# Problèmes électriques
electrical_replacement_prices = {
    "remplacement du panneau électrique": (879, 2747.25),  # €
    "réparation ou remplacement des fils et des câbles": (546, 1638),  # €
    "installation de nouveaux circuits électriques": (546, 1638),  # €
    "installation de systèmes d'éclairage": (227, 1137)  # € par pièce
}

electrical_upgrade_prices = {
    "mise à niveau des prises de courant et des interrupteurs": (72.50, 108.75),  # € par prise ou interrupteur
    "mise à jour des systèmes de sécurité": (546, 2185)  # €
}

# Problèmes de plomberie
bathroom_prices = {
    "lavabo": 60.06,  # €
    "baignoire/douche": 150.15,  # €
    "toilette standard": 200,  # €
    "toilette haut de gamme": 600,  # €
    "robinet de douche": (60, 120),  # €
    "robinet de lavabo": (30, 100)  # €
}

plumbing_repair_prices = {
    "réparation d'une fuite mineure": (150, 250),  # €
    "réparation d'une fuite importante": (250, 550),  # €
    "remplacement d'un tuyau": (100, 200),  # €
    "remplacement d'un robinet": (100, 250),  # €
    "débouchage d'un évier": (100, 200),  # €
    "remplacement d'un évier": (200, 500)  # €
}

# Problème de chauffage, ventilation, climatisation
hvac_maintenance_prices = {
    "entretien annuel d'une chaudière": (120, 200),  # €
    "entretien du système électronique": 74.62,  # €
    "entretien d'une pompe à chaleur": (100, 300),  # €
    "nettoyage des conduits de ventilation": (200, 500)  # €
}

hvac_replacement_repair_prices = {
    "ventilation + conduits": 546,  # €
    "chaudière standard": (1500, 3000),  # €
    "chaudière haut de gamme": (4000, 7000),  # €
    "installation d'un nouveau système de chauffage (pompe à chaleur)": (5000, 12000),  # €
    "réparation ou remplacement de la climatisation": (1000, 5000),  # €
    "installation d'un système de ventilation mécanique contrôlée (VMC)": (1000, 4000),  # €
    "installation d'un système de climatisation centralisée": (3000, 8000)  # €
}

# Problème d'isolation
insulation_prices = {
    "isolation des combles": (20, 80),  # € par mètre carré
    "isolation des murs": (50, 150),  # € par mètre carré
    "isolation des planchers": (30, 80),  # € par mètre carré
    "installation de fenêtres isolantes": (300, 1000),  # € par fenêtre
    "isolation des portes": (50, 500)  # € par porte
}

# Présence de nuisibles
pest_control_prices = {
    "élimination de souris et de rats": (150, 500),  # €
    "élimination de punaises de lit": (200, 1000),  # €
    "élimination des guêpes et des nids de frelons": (60, 250),  # €
    "élimination des fourmis": (70, 200)  # €
}

pest_damage_repair_prices = {
    "réparation des dommages causés par les rongeurs": (150, 500),  # €
    "réparation des dommages causés par les insectes": (200, 1000),  # €
    "nettoyage et désinfection de la zone touchée": (50, 100)  # € par mètre carré
}

# Problèmes d'humidité
humidity_detection_treatment_prices = {
"diagnostic de l'humidité": (80, 150), # €
"traitement de l'humidité par injection": (100, 150), # € par mètre linéaire
"traitement de l'humidité par ventilation": (1500, 5000), # €
"traitement de l'humidité par drainage": (1000, 5000) # €
}

#Problèmes de toiture
roof_repair_prices = {
"remplacement de tuiles": (2, 4), # € par tuile
"réparation de fuite": (200, 400), # €
"remplacement du revêtement de toit": (40, 100), # € par mètre carré
"nettoyage de la toiture": (200, 500) # €
}

roof_replacement_prices = {
"toiture en tuiles": (60, 150), # € par mètre carré
"toiture en ardoise": (80, 250), # € par mètre carré
"toiture en zinc": (120, 200), # € par mètre carré
"toiture en bac acier": (30, 100) # € par mètre carré
}

#Problèmes de fondation
foundation_repair_prices = {
"réparation de fissures - injection de résine époxy": (100, 300), # € par fissure
"ancrage de la fondation": (2000, 5000) # €
}

foundation_settlement_repair_prices = {
"injection de mousse polyuréthane": (1000, 3000), # €
"remplacement des pieux": (3000, 7000) # €
}

foundation_wall_repair_prices = {
"rénovation de mur en blocs de béton": (100, 150), # € par mètre carré
"réparation de murs de fondation en briques": (100, 250), # € par mètre carré
"rénovation de mur en béton coulé": (300, 500) # € par mètre carré
}
